# CIL Project 3 - Road Segmentaion

### Imports

In [7]:
import os
import sys

import time
import utils
import glob

import math
import numpy as np

from typing import Optional

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torch.optim.lr_scheduler as lr

from torchvision import transforms
from torchvision.datasets import MNIST

import torchmetrics as tm


import pytorch_lightning as pl

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer, seed_everything


### Fixing seed for Reproducability

In [8]:
seed = 121
seed_everything(seed, workers=True)

Global seed set to 121


121

### Config setup

In [9]:
# move out of repo and then into the cil data folder
data_dir = "../cil_data/"

In [10]:
batch_size = 16
bs_train = batch_size
bs_eval = batch_size

learning_rate = 0.005

data_workers = 8

### Data Transformations and Loading

In [11]:
class DataModule(pl.LightningDataModule):   
    def __init__(self, data_dir: str = './'):
        super().__init__()
        
        self.data_dir = data_dir
        
        self.train_transform = transforms.Compose(
            [
                
            ]
        )
        
        self.valid_transform = transforms.Compose(
            [
                
            ]
        )

        self.test_transform = transforms.Compose(
            [
                
            ]
        )

    def setup(self, stage: Optional[str] = None):  
        if stage in (None, 'fit'):
            self.train_data = Dataset(os.path.join(self.data_dir, "training"), transform=self.train_transform)
            self.valid_data = Dataset(os.path.join(self.data_dir, "validation"), transform=self.valid_transform)

        if stage in (None, 'test'):
            self.test_data = Dataset(os.path.join(self.data_dir, "test"), transform=self.test_transform, is_test=True)

    def train_dataloader(self):
        return DataLoader(
            self.train_data,
            batch_size=bs_train,
            shuffle=True,
            num_workers=data_workers,
            drop_last=True,
            pin_memory=True
        )

    def val_dataloader(self):
        return DataLoader(
            self.valid_data,
            batch_size=bs_eval,
            shuffle=False,
            num_workers=data_workers,
            drop_last=True,
            pin_memory=True
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_data,
            batch_size=bs_eval,
            shuffle=False,
            num_workers=data_workers,
            pin_memory=True
        )

In [12]:
data = DataModule(data_dir)
data.setup()

NameError: name 'Dataset' is not defined

### Model and System Definition

In [13]:
class LightningSimpleSystem(pl.LightningModule):
    def __init__(self, model, datamodule: pl.LightningDataModule=None, batch_size=batch_size, lr=learning_rate):
        super().__init__()
        
        self.model = model
        
        self.datamodule = datamodule
        
        self.batch_size = batch_size
        self.lr = lr
        
        #self.me = MetricsEngine(C.METRIC_TARGET_LENGTHS)
        #self.me.reset()

    def training_step(self, batch, batch_idx):
        X, y = batch
        
        y_pred = self.model(X)
       
        loss = tm.functional.mean_squared_error(y_pred, y)
        
        self.log('mse', loss)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        X, y = batch
        
        y_pred = self.model(X)
       
        loss = self.calculate_metric(y_pred, y)
        
        self.log('val_loss', loss)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        X, seq_ids = batch
        
        y_pred = self.model(X)
        
        return dict(list(zip(seq_ids, list(zip(y_pred.detach().cpu().numpy(), X.detach().cpu().numpy())))))
    
    def test_epoch_end(self, outputs):
        results = {k: v for d in outputs for k, v in d.items()}
        
        self.test_results = results
    
    def calculate_metric(self, y_pred, y):
        self.me.reset()
        self.me.compute_and_aggregate(y_pred, y)
        
        return system.me.get_final_metrics()['joint_angle'].mean() * 24

    def train_dataloader(self):
        return self.datamodule.train_dataloader()

    def val_dataloader(self):
        return self.datamodule.val_dataloader()

    def test_dataloader(self):
        return self.datamodule.test_dataloader()
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = lr.ReduceLROnPlateau(optimizer, patience=4, verbose=2)
        
        return {
            'optimizer': optimizer,
            'lr_scheduler': scheduler,
            'monitor': 'val_loss'
        }

In [14]:
class GraphConvolution(nn.Module):
    def __init__(self, feature_size_in=135, feature_size_out=540, bias=True, n_nodes=135):
        super(GraphConvolution, self).__init__()
        
        self.feature_size_in = feature_size_in
        self.feature_size_out = feature_size_out
        
        self.weights = nn.Parameter(torch.FloatTensor(feature_size_in, feature_size_out))
        self.attention = nn.Parameter(torch.FloatTensor(n_nodes, n_nodes))
        
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(feature_size_out))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weights.size(1))
        self.weights.data.uniform_(-stdv, stdv)
        self.attention.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
    
    
    def forward(self, input):
        support = torch.matmul(input, self.weights)
        output = torch.matmul(self.attention, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

        
    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


In [15]:
class GCN(nn.Module):
    def __init__(self, input_features, hidden_features, p_dropout, n_nodes=135):
        super(GCN, self).__init__()
        
        self.gc1 = GraphConvolution(input_features, hidden_features, n_nodes=n_nodes)
        self.bn1 = nn.BatchNorm1d(n_nodes * hidden_features)
        
        self.gc2 = GraphConvolution(hidden_features, hidden_features, n_nodes=n_nodes)
        self.bn2 = nn.BatchNorm1d(n_nodes * hidden_features)
        
        self.gc3 = GraphConvolution(hidden_features, input_features, n_nodes=n_nodes)

        self.do = nn.Dropout(p_dropout)
        self.ac = nn.Tanh()
        
    def forward(self, x):
        y = self.gc1(x)
        b, n, f = y.shape
        y = self.bn1(y.view(b, -1)).view(b, n, f)
        y = self.ac(y)
        y = self.do(y)
        
        y = self.gc2(y)
        b, n, f = y.shape
        y = self.bn2(y.view(b, -1)).view(b, n, f)
        y = self.ac(y)
        y = self.do(y)

        y = self.gc3(y)
        y = y + x

        return y

In [16]:
model_name = "SequentialModel"

class Encoder(nn.Module):
    def __init__(self,input_size,hidden_size,drop):
        super().__init__()
        
        self.rnn = nn.GRU(input_size,
                          hidden_size,
                          1,
                          #bidirectional = True,
                          batch_first=True, dropout=drop
                         )
        self.fc = nn.Linear(hidden_size,hidden_size)
        
        
    def forward(self, X):
        
        dct_m, idct_m = get_dct_matrix(3)
        dct_m = torch.from_numpy(dct_m).float().cuda()
        idct_m = torch.from_numpy(idct_m).float().cuda()
        
        src_value_tmp = X.reshape(-1,3,3)
        src_value_tmp = torch.matmul(dct_m[:8].unsqueeze(dim=0),src_value_tmp).reshape(X.shape)
        #out, hidden = self.rnn(X)
        out, hidden = self.rnn(src_value_tmp)
        hidden = hidden.transpose(0,1)
        hidden = self.fc(hidden)
        hidden = hidden.transpose(0,1)

        return out, hidden
    
class Decoder(nn.Module):
    def __init__(self,input_size,hidden_size,drop):
        super().__init__()
        
        self.rnn = nn.GRU(input_size,
                          hidden_size,
                          1,
                          batch_first=True, dropout=drop
                         )
        self.fc_out = nn.Linear(hidden_size,hidden_size)
        
    def forward(self, X, hidden_in):
        
        out, hidden_out = self.rnn(X, hidden_in)
        output = torch.zeros_like(out).cuda()
        output[:,-1,:] = self.fc_out(out[:,-1,:])
                
        return output, hidden_out
    

class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.attn = nn.Linear(enc_hid_dim + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1)
        
    def forward(self, hidden, encoder_outputs):
        enc_hid_dim = hidden.shape[2]
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        
        before_hidden = hidden
        out_hidden = torch.zeros(src_len, batch_size, enc_hid_dim).cuda()
        """
        for i in range(src_len):
            out_hidden[i,:,:] = hidden[0,:,:]
        """
        out_hidden[:,:,:] = hidden[0,:,:]

        before_hidden = torch.cat((before_hidden,before_hidden),dim=0)

        
        out_hidden = out_hidden.transpose(0,1)
        
        to_att = torch.cat((out_hidden, encoder_outputs), dim = 2)
        
        energy = torch.tanh(self.attn(torch.cat((out_hidden, encoder_outputs), dim = 2))) 
        

        a = self.v(energy).squeeze(2)
        
        
        return F.softmax(a, dim=1)    


class AttentionDecoder(nn.Module):
    def __init__(self,input_size,hidden_size,drop,attention):
        super().__init__()
        self.attention = attention
        
        self.rnn = nn.GRU(input_size+hidden_size,
                          hidden_size,
                          1,
                          batch_first=True, dropout=drop
                         )
        self.fc_out = nn.Linear(hidden_size*3,input_size)
        
    def forward(self, X, hidden_in,encoder_outputs):
        
        a = self.attention(hidden_in,encoder_outputs)
        
        a = a.unsqueeze(1)
        
        weighted = torch.bmm(a,encoder_outputs)
        
        rnn_input = torch.cat((X,weighted),dim=2)
        
        out, hidden_out = self.rnn(rnn_input, hidden_in)

        
        X = X.squeeze(0)
        out = out.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = torch.zeros_like(out).cuda()


        to_fc2 = torch.cat((out,weighted,X),dim=2)

        to_input = to_fc2.squeeze(1)

       
        output[:,-1,:] = self.fc_out(to_input)
                              
        return output, hidden_out
      
    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder,output_seq_length):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
        self.gcn = GCN(input_features=144*2, hidden_features=512, p_dropout=0.3, n_nodes=135)
        
        self.output_seq_length = output_seq_length
        
        
    def forward(self,X):
        
        #print(X.shape)

        outputs = torch.zeros_like(X[:,:self.output_seq_length,:])
        
        encoder_out, hidden = self.encoder(X)
        
        #print("encoder_out shape: ", encoder_out.shape)
        #print("encoder out hidden shape: ", hidden.shape)
        
        input = X[:,-1,:].unsqueeze(1)
        
        for i in range(24):
            
            output, hidden_out = self.decoder(input, hidden ,encoder_outputs=encoder_out)
            
            outputs[:,i,:] = output[:,-1,:]
            
            input = output
            hidden = hidden_out
    
        dct_m, idct_m = get_dct_matrix(3)
        dct_m = torch.from_numpy(dct_m).float().cuda()
        idct_m = torch.from_numpy(idct_m).float().cuda()
        
        dct_gcn, idct_gcn = get_dct_matrix(144)
        dct_gcn = torch.from_numpy(dct_gcn).float().cuda()
        idct_gcn = torch.from_numpy(idct_gcn).float().cuda()
        
        out_33 = outputs.reshape(-1,3,3)
        back_out = torch.matmul(idct_m[:8].unsqueeze(dim=0), out_33).reshape(outputs.shape)
        
        #print("X: ", X.shape)
        #print("back_outs: ", back_out.shape)
        last_el = X[:, -1, :]
        #print("last eel: ", last_el.shape)
        padded = torch.cat((X, last_el.unsqueeze(1).repeat(1, 24, 1)), 1)
        #print("padded:", padded.shape)
        
        concat_out = torch.cat((X, back_out), 1)
        #print("concat_out: ", concat_out.shape)
        
        
        # to dct
        
        padded_dct = torch.matmul(dct_gcn[:].unsqueeze(dim=0), padded).transpose(1,2)
        
        out_dct = torch.matmul(dct_gcn[:].unsqueeze(dim=0), concat_out).transpose(1,2)
        gcn_in = torch.cat([padded_dct, out_dct], dim=-1)
        
        gcn_output = self.gcn(gcn_in)
        output_restored = torch.matmul(idct_gcn[:, :144].unsqueeze(dim=0), gcn_output[:, :, :144].transpose(1,2))
        #print("restored outp:", output_restored.shape)
    
      
        return output_restored[:, -24:, :]


    
encoder = Encoder(input_size=135,hidden_size=135,drop=0.2)
attention_model = Attention(enc_hid_dim=135,dec_hid_dim=135)
decoder = AttentionDecoder(input_size=135,hidden_size=135,drop=0.2,attention=attention_model)
model = Seq2Seq(encoder,decoder,output_seq_length=24)

/home/cupcake/.local/lib/python3.9/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [19]:
system = LightningSimpleSystem(model, data)

In [20]:
X, y = next(iter(system.train_dataloader()))

AttributeError: 'DataModule' object has no attribute 'train_data'

### Setup saving environment

Can we make it nicer than just having the time as the name?
Somehow increase the experiment id everytime we save it?
Perhaps when running a test:  
   - See in directory which experiment id was last
   - Increast that id by 1
   - Save model parameters, prediction in the corresponding folder

In [13]:
experiment_id = int(time.time())
experiment_name = model_name
model_dir = utils.create_model_dir(C.EXPERIMENT_DIR, experiment_id, experiment_name)

In [14]:
code_files = glob.glob('./*.py', recursive=False)
utils.export_code(code_files, os.path.join(model_dir, 'code.zip'))
config.to_json(os.path.join(model_dir, 'config.json'))  ## parameters form configuration.py
# The parameters can change since we use pytorch lightning

cmd = sys.argv[0] + ' ' + ' '.join(sys.argv[1:])
with open(os.path.join(model_dir, 'cmd.txt'), 'w') as f:
    f.write(cmd)

### Train model

In [15]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   patience=15,
   verbose=2,
   mode='min'
)

In [16]:
trainer = pl.Trainer(
    deterministic=True,
    #fast_dev_run=True,
    gpus=-1, 
    auto_select_gpus=True, 
    #auto_lr_find=True, 
    #lr=0.00001,
    #benchmark=True,    
    progress_bar_refresh_rate=10,
    stochastic_weight_avg=True, 
    auto_scale_batch_size='binsearch',
    #limit_train_batches=32, #batch size is 16
    callbacks=[early_stop_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [17]:
trainer.tune(system)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 666
Batch size 2 succeeded, trying batch size 4
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 666
Batch size 4 succeeded, trying batch size 8
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 666
Batch size 8 succeeded, trying batch size 16
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 666
Batch size 16 succeeded, trying batch size 32
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 666
Batch size 32 succeeded, trying batch size 64
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 666
Batch size 64 succeeded, trying batch size 128
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 666
Batch size 128 succeeded, trying batch size 256
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 666
Batch size 256 succeeded, trying batch size 512
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 666
Batch size 512 succeeded, trying batch size 102

{'scale_batch_size': 4170}

In [18]:
trainer.fit(system)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | Seq2Seq | 1.3 M 
----------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.098     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 666


Training: 2it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 15 records. Best score: 2.870. Signaling Trainer to stop.


Epoch    46: reducing learning rate of group 0 to 1.5849e-09.


### Save Model Weights

In [19]:
torch.save(model.state_dict(), os.path.join(model_dir, 'saved_model_state.pth'))

### Predict and save in right format

In [20]:
trainer.test(system)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

In [21]:
results = system.test_results
len(results.keys())

678

In [22]:
fname = 'predictions_in{}_out{}.csv'.format(config.seed_seq_len, config.target_seq_len)
evaluate._export_results(results, os.path.join(model_dir, fname))

In [23]:
print(model_dir)

/cluster/home/vvisuval/experiments/1624177738-SequentialModel
